# Analisis de datos sobre la evolución del anime

### Comenzamos con la obtención de el data set

In [ ]:
import os
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import time
df = pd.read_json('data.json')

### Exploración inicial

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.describe(include='object').T

In [ ]:
# No me deja ver los valores unicos de todas las columnas porque las columnas contienen listas.

In [ ]:
for col in df.select_dtypes(include='object').columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        print(f"Columna {col} contiene listas")

In [ ]:
# Convertir las listas de genres en columnas separadas
# Primero, creamos una nueva DataFrame con las listas convertidas en series.
genres_split = df['genres'].apply(lambda x: pd.Series(x) if isinstance(x, list) else pd.Series([None]*3))
genres_split.columns = ['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5','genre_6','genre_7']

# Concatenar las nuevas columnas al DataFrame original
df_nuevo = pd.concat([df.drop(columns=['genres']), genres_split], axis=1)

In [ ]:
# Calcular la proporción de valores nulos en las columnas de géneros para ver con cuales me quedo.
proporcion_nulos = df_nuevo[['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7']].isna().sum() / df_nuevo.shape[0]
print(proporcion_nulos)

In [ ]:
df_nuevo = df_nuevo.drop(['genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7'], axis=1)
df = df_nuevo

In [ ]:
#Me quedo solo con el primer theme.
def extraer_theme_antes_coma(themes):
    if isinstance(themes, list) and len(themes) > 0:
        return themes[0].split(',')[0].strip()
    return None

df['themes'] = df['themes'].apply(extraer_theme_antes_coma)

In [ ]:
#Saco de la lista los titulos.
df['titles'] = df['titles'].apply(lambda x: x[0] if isinstance(x, list) and x else x)

In [ ]:
df.nunique()

In [ ]:
df.columns

In [ ]:
#Al final themes no me ha parecido interesante.
df.rename(columns={'popularity':'audicence_rank'}, inplace=True)
df.drop(['favorites','themes'], axis=1)

In [ ]:
df = df[['mal_id', 'titles', 'type', 'genre_1', 'genre_2', 'source', 'studios', 'episodes', 'rating', 'score',
       'audicence_rank','scored_by', 'rank', 'members', 'favorites',
       'synopsis']]

In [ ]:
# Ahora que tengo los datos un poco ordenados y se que me falta, voy a hacer web scraping

### Web sraping

In [ ]:
#Primero las fechas
animes = []

for offset in range(0, 5550, 50):
    url = f"https://myanimelist.net/topanime.php?limit={offset}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    lista = soup.find_all('tr', class_='ranking-list')

    for anime in lista:
        title_tag = anime.find('h3', class_='anime_ranking_h3')
        title = title_tag.text.strip() if title_tag else None

        
        info_div = anime.find('div', class_='information di-ib mt4')
        fecha_emision = None
        if info_div:
            lines = info_div.text.strip().split('\n')
            if len(lines) >= 2:
                fecha_emision = lines[1].strip()

        animes.append({
            'title': title,
            'fecha_emision': fecha_emision
        })

    time.sleep(1)  

df_fechas = pd.DataFrame(animes)
df_fechas.head()


### API

In [ ]:
def get_popular_animes(limit=5550):
    url = 'https://api.jikan.moe/v4/anime'
    animes = []
    page = 1
    per_page = 25

    while len(animes) < limit:
        response = requests.get(url, params={'page': page, 'limit': per_page})
        print(f"Página {page}, Código: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            if not data.get('data'):
                print("No hay más datos.")
                break

            animes.extend(data['data'])
            print(f"Total recolectado: {len(animes)}")
            page += 1
            time.sleep(1)  # 🛑 Espera obligatoria para evitar rate limit
        elif response.status_code == 429:
            print("⚠️ Rate limit alcanzado. Esperando 10 segundos...")
            time.sleep(10)  # Espera más larga para recuperarse
        else:
            print(f"❌ Error en página {page}: {response.text}")
            break

    return animes

# Ejecutar
popular_animes = get_popular_animes()

In [ ]:
#Me ha gustado más lo que he sacado de la api.
df_state = pd.DataFrame(popular_animes)
df_state.columns

In [ ]:
columns_to_keep = ['mal_id', 'status', 'airing', 'season', 'year']
df_2 = df_state[columns_to_keep]

In [ ]:
df_2.head()

### Voy a juntar la información que he sacado

In [ ]:
df_final = pd.merge(df, df_2, on = 'mal_id', how='left')

In [ ]:
df_final.head()

#### He pasado este df a un csv que esta en la misma carpeta, y que usare para hacer la limpieza y los insights. Hasta aqui la minería para conseguir la informacion que queria.